In [ ]:
import os
from dotenv import load_dotenv
from cognite.client import CogniteClient, ClientConfig
from cognite.client.credentials import OAuthClientCredentials

# Load the values from the .env file
load_dotenv()

# Get all environment variables
env_vars = os.environ

project_name = os.getenv("COGNITE_PROJECT")
mqtt_broker = os.getenv("MQTT_BROKER")
mqtt_port = int(os.getenv("MQTT_PORT", 1883))

prefix_name = os.getenv("NAMING_NAME")
prefix_project = os.getenv("NAMING_PROJECT")

In [ ]:
oauth_provider = OAuthClientCredentials(
    token_url=os.getenv("TOKEN_URL"),
    client_id=os.getenv("GM_CLIENT_ID"),
    client_secret=os.getenv("GM_CLIENT_SECRET"),
    scopes=[os.getenv("SCOPES")],
    # Any additional IDP-specific token args. e.g.
    # audience="some-audience"
)

clientConfig = ClientConfig(
    client_name=os.getenv("CLIENT_NAME"),
    project=project_name,
    credentials=oauth_provider,
    base_url=os.getenv("BASE_URL"),
    debug=False,
)

client = CogniteClient(clientConfig)

In [ ]:
from duckduckgo_search import ddg_images
from fastdownload import download_url
from fastcore.all import *
from fastai.vision.all import *
from contextlib import contextmanager
import pathlib

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
term = "coffee"
file_path = Path(str('images/'+term+'.jpg'))
file_name = term+'.jpg'

download_url(search_images('images/'+term+' photos', max_images=1)[0], 'images/'+term+'.jpg', show_progress=True)
Image.open('images/'+term+'.jpg').to_thumb(256,256)

In [ ]:
from cognite.client.data_classes.files import FileMetadata

# Create FileMetadata object with the specified metadata
metadata = {
    "classify": "true"
}

# Upload a new file
with open(file_path, "rb") as f:
    uploaded_file = client.files.upload(file_path, name=file_name, metadata=metadata)

print(uploaded_file)


In [ ]:
@contextmanager
def set_posix_windows():
    posix_backup = pathlib.PosixPath
    try:
        pathlib.PosixPath = pathlib.WindowsPath
        yield
    finally:
        pathlib.PosixPath = posix_backup

In [ ]:
model_path = pathlib.Path('models/model.pkl')

with set_posix_windows():
    learn = load_learner(model_path)

In [24]:
from cognite.client.data_classes import FileMetadataUpdate

# Define the metadata filter
metadata_filter = {
    "classify": "true"
}

# Retrieve files with the specified metadata filter
files_with_classify_true = client.files.list(metadata=metadata_filter)

# Print the file names and IDs
for file in files_with_classify_true:
    file_metadata = client.files.retrieve(id=file.id)
    file_content = client.files.download_bytes(id=file.id)

    download_path = "fromcdf/"+file_metadata.name

    with open(download_path, "wb") as f:
        f.write(file_content)

    type_of_beverage,_,probs = learn.predict(download_path)

    # Define the new metadata
    new_metadata = {
        "beverage": type_of_beverage  # Add a new metadata key-value pair
    }

    my_update = FileMetadataUpdate(id=file.id).metadata.add({"beverage": type_of_beverage})

    # Update the file with the new metadata
    updated_file = client.files.update(my_update)

    # Print the updated file metadata
    print("Updated metadata:", updated_file.metadata)

    os.remove(download_path)


  

Updated metadata: {'beverage': 'beer', 'classify': 'true'}


Updated metadata: {'beverage': 'coffee', 'classify': 'true'}
